In [1]:
import torchvision
import torch
import os

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

/Users/chuan/miniforge3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import torch
import math
# this ensures that the current MacOS version is at least 12.3+
print(torch.backends.mps.is_available())
# this ensures that the current current PyTorch installation was built with MPS activated.
print(torch.backends.mps.is_built())

dtype = torch.float
device = torch.device("mps")

# Create random input and output data
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Randomly initialize weights
a = torch.randn((), device=device, dtype=dtype)
b = torch.randn((), device=device, dtype=dtype)
c = torch.randn((), device=device, dtype=dtype)
d = torch.randn((), device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)

# Backprop to compute gradients of a, b, c, d with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # Update weights using gradient descent
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d


print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

True
True
99 2036.2734375
199 1404.9825439453125
299 971.3155517578125
399 673.0686645507812
499 467.72637939453125
599 326.1922607421875
699 228.53268432617188
799 161.0756072998047
899 114.43142700195312
999 82.145751953125
1099 59.776222229003906
1199 44.26213073730469
1299 33.492340087890625
1399 26.009098052978516
1499 20.804893493652344
1599 17.182479858398438
1699 14.658984184265137
1799 12.89959716796875
1899 11.672037124633789
1999 10.814900398254395
Result: y = -0.04334169626235962 + 0.8393216729164124 x + 0.007477166596800089 x^2 + -0.09085267782211304 x^3


In [ ]:
to_pil = torchvision.transforms.ToPILImage()
class COCOdataset(Dataset):
    def __init__(self, root, img_paths, label_paths, img_size=640):
        self.root_img = root
        self.root_label = root.replace("images", "labels")
        self.img_paths = img_paths
        self.label_paths = label_paths
        self.resize = transforms.Resize((img_size, img_size))
        self.to_tensor = transforms.ToTensor()

    def __len__(self):
        # Return the number of images and labels in the dataset
        assert len(self.img_paths) == len(self.label_paths), "There is an unmatched number of images and labels!"
        return len(self.img_paths)
    
    def __getitem__(self, idx):
        # Get the image and label paths at the given index
        image_path = self.img_paths[idx]
        label_path = self.label_paths[idx]
        assert image_path.rstrip("jpg") == label_path.rstrip("txt"), "Image and label does not match"

        with open(os.path.join(self.root_label, label_path)) as f:
            label = np.loadtxt(f)
            label = torch.from_numpy(label).float()
            if label.dim() == 1:
                label = label.unsqueeze(0)
            
        # image = torchvision.io.read_image(os.path.join(self.root_img, image_path))
        image = Image.open(os.path.join(self.root_img, image_path))
        image, label = self.pad_and_scale(image, label)
        image = self.to_tensor(image)
        label = self.pad_lab(label)
        return to_pil(image), label, [os.path.join(self.root_img, image_path),os.path.join(self.root_label, label_path)]
    
    def pad_and_scale(self, img, lab):
        w,h = img.size
        if w==h:
            padded_img = img
        else:
            dim_to_pad = 1 if w<h else 2
            if dim_to_pad == 1:
                padding = (h - w) / 2
                padded_img = Image.new('RGB', (h,h), color=(127,127,127))
                padded_img.paste(img, (int(padding), 0))
                lab[:, [1]] = (lab[:, [1]] * w + padding) / h
                lab[:, [3]] = (lab[:, [3]] * w / h)
            else:
                padding = (w - h) / 2
                padded_img = Image.new('RGB', (w, w), color=(127,127,127))
                padded_img.paste(img, (0, int(padding)))
                lab[:, [2]] = (lab[:, [2]] * h + padding) / w
                lab[:, [4]] = (lab[:, [4]] * h  / w)
        
        padded_img = self.resize(padded_img)
        return padded_img, lab

    def pad_lab(self, lab):
        pad_size = 80 - lab.shape[0]
        if(pad_size>0):
            padded_lab = F.pad(lab, (0, 0, 0, pad_size), value=1)
        else:
            padded_lab = lab
        return padded_lab

# Create the dataset
root = "../datasets/car/images/train2017/"
img_paths = os.listdir(root)
label_paths = os.listdir(root.replace("images", "labels"))
dataset = COCOdataset(root, sorted(img_paths), sorted(label_paths), img_size=640)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', autoshape=False)

img_with_car_no_detected = []
for idx, (img, label, image_path) in enumerate(dataset):
    res = model(img)